In [1]:
#setup

import io
import os
import re
import shutil
import string
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print('gpu', gpu)
    tf.config.experimental.set_memory_growth(gpu, True)
    print('memory growth:' , tf.config.experimental.get_memory_growth(gpu))

from datetime import datetime
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

gpu PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
memory growth: True


In [3]:
#download the dataset

url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("/home/dawidkubicki/Datasets/aclImdb_v1.tar.gz", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

84131840/84125825 [==============================] - 27s 0us/step


FileNotFoundError: [Errno 2] No such file or directory: '/home/dawidkubicki/Datasets/aclImdb'

In [9]:
dataset_dir = os.path.dirname("/home/dawidkubicki/Datasets/aclImdb")
os.listdir(dataset_dir)

['test', 'imdbEr.txt', 'imdb.vocab', 'README', 'train']

In [10]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['unsupBow.feat',
 'neg',
 'urls_unsup.txt',
 'pos',
 'urls_pos.txt',
 'urls_neg.txt',
 'unsup',
 'labeledBow.feat']

In [11]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [12]:
batch_size = 1024
seed = 123
train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2, 
    subset='training', seed=seed)
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2, 
    subset='validation', seed=seed)

Found 75000 files belonging to 3 classes.
Using 60000 files for training.
Found 75000 files belonging to 3 classes.
Using 15000 files for validation.


In [13]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(5):
    print(label_batch[i].numpy(), text_batch.numpy()[i])

1 b"Ask yourself where she got the gun? Remember what she was taught about the mark's mindset when the con is over? The gun had blanks and it was provided to her from the very beginning.<br /><br />When the patient comes back at the end she was SUPPOSED to see him drive away in the red convertible and lead her to the gang splitting up her 80 thousand.<br /><br />The patient was in on the con from the beginning.<br /><br />Mantegna does not die in the end - the gun had blanks.<br /><br />There - enough spoilers for you there? This is why people are giving it such high ratings. It's extremely original because of the hidden ending and how it cons MOST of the audience."
2 b"For some reason, people seem to have a problem differentiating this movie from Trail of the Pink Panther.<br /><br />At any rate, this work does nothing but serve to remind us how sad the world is without Peter Sellers in it.<br /><br />They brought back the same old favorites from Trail (Dreyfus, Cato, Litton, etc.), b